# Fine Tune Stable Diffusion

In [ ]:
# change to content directory
!cd /content/
# clone huggingface diffuseres repositury from github
!git clone https://github.com/huggingface/diffusers.git
!pip install ./diffusers
# installing requirements for diffuers
!pip install -U -r /content/diffusers/examples/text_to_image/requirements.txt

In [ ]:
# cheking the gpu
!nvidia-smi

In [ ]:
!accelerate config default --mixed_precision fp16

#### directory to save model after training

In [ ]:
import os

# 3 stable diffusion model
os.environ['MODEL_NAME'] = f'CompVis/stable-diffusion-v1-2'
# dataset
os.environ['DATASET_NAME'] = f'choudharyprince890/martin_valen_hoodies'
# directory to save model
os.environ['OUTPUT_DIR'] = f'sd_martin_valen-model-v1-2_400'

add token for authentication

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

### model parameters

https://github.com/huggingface/diffusers/blob/main/examples/text_to_image/README.md

In [ ]:
!accelerate launch diffusers/examples/text_to_image/train_text_to_image.py \
  --pretrained_model_name_or_path=$MODEL_NAME \
  --dataset_name=$DATASET_NAME \
  --use_ema \
  --resolution=512 --center_crop --random_flip \
  --train_batch_size=1 \
  --gradient_accumulation_steps=4 \
  --gradient_checkpointing \
  --mixed_precision="fp16" \
  --max_train_steps=400 \
  --learning_rate=1e-05 \
  --max_grad_norm=1 \
  --push_to_hub \
  --checkpointing_steps=100000 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --output_dir=$OUTPUT_DIR

In [ ]:
# Stable Diffusion V1
from diffusers import StableDiffusionPipeline
import torch
from PIL import Image

model_path = "./sd_martin_valen-model-v1-2_400_demo"
pipe = StableDiffusionPipeline.from_pretrained(model_path, torch_dtype=torch.float16)
pipe.to("cuda")

# Run inference using ChatGPT prompts to acquire 4 image panels
image1 = pipe(prompt="black hoodie with a front half zipper by martin valen").images[0]
image1.save("ProductSnapAI_panel_1.png")

image2 = pipe(prompt="white hoodie with a blue design by martin valen").images[0]
image2.save("ProductSnapAI_panel_2.png")

image3 = pipe(prompt="stripped hoodie by martin valen").images[0]
image3.save("ProductSnapAI_panel_3.png")

image4 = pipe(prompt="camouflage hoodie by martin valen").images[0]
image4.save("ProductSnapAI_panel_4.png")

# Image grid helper function from HuggingFace
def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size

    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid

all_images = [image1, image2, image3, image4]
grid = image_grid(all_images, rows=1, cols=4)
grid